In [1]:
import pandas as pd
import os
import zipfile

# Extracción de archivos #

Primero, extraeremos solo los productos del database que nos sirven para nuestra investigación en curso.

In [2]:
zip_path = "datos-covid-19.zip"
extract_path = "datos_covid_extraccion"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    all_files = zip_ref.namelist()
    productos = [
        "producto1", "producto34", "producto38", "producto41",
        "producto65", "producto66", "producto80", "producto84"
    ]
    
    archivos_a_extraer = []
    
    for producto in productos:
        patrones = [
            f"{producto}/",
            f"output/{producto}/", 
            f"datos-covid-19/{producto}/",
            f"datos-covid-19/output/{producto}/"
        ]
        
        for archivo in all_files:
            if any(patron in archivo for patron in patrones) and archivo.endswith('.csv'):
                archivos_a_extraer.append(archivo)        
    if not os.path.exists(extract_path):
        os.makedirs(extract_path)
    for archivo in archivos_a_extraer:
        try:
            zip_ref.extract(archivo, extract_path)
        except Exception as e:
            print('upis')

# Limpieza de archivos #

En esta sección iremos limpiando los datos para dejar los de la Región Metropolitana, por cada producto ya leído se armará un DataFrame con lo escencial para luego concatenar todo por comunas.

In [3]:
# Producto 1: Se filtrará por los datos de la Región Metropolitana, dejando la cantidad de casos confirmados por año, por cada región.

csv_path = "datos_covid_extraccion/datos-covid-19/producto1/Covid-19_std.csv"
df = pd.read_csv(csv_path)
df_rm = df[df["Region"] == "Metropolitana"].copy()
df_rm["Fecha"] = pd.to_datetime(df_rm["Fecha"], dayfirst=True)
df_rm["Año"] = df_rm["Fecha"].dt.year
df_agrupado = df_rm.groupby(["Comuna", "Año"])["Casos confirmados"].sum().reset_index()
df_agrupado.to_csv('casos_cofirmados_comuna.csv', index=False)
df_agrupado.head(20)


,Comuna,Año,Casos confirmados
0,Alhue,2020,16778.0
1,Alhue,2021,70930.0
2,Alhue,2022,191187.0
3,Alhue,2023,5597.0
4,Buin,2020,160555.0
5,Buin,2021,898107.0
6,Buin,2022,2308531.0
7,Buin,2023,58051.0
8,Calera de Tango,2020,35802.0
9,Calera de Tango,2021,203809.0


In [4]:
# Producto 34: Cuarentenas según manzanas de censo 2017 (con códigos por manzana)

df_cuar = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto34/Cuarentenas_Censo17.csv")

def limpiar_nombre_comuna(nombre):
    nombre = nombre.strip()
    nombre = nombre.split('(')[0].split('[')[0]
    for s in ['Extensión a Total', 'Extendido', 'Re-ingreso total', 'Re-ingreso', 
              'Nororiente', 'Poniente', 'Norte', 'Sur', '(Área Urbana)']:
        nombre = nombre.replace(s, '')
    nombre = ' '.join(nombre.split())
    nombre = nombre.replace('Macúl', 'Macul').replace('TilTil', 'Til Til')
    return nombre.strip()

df_cuar['Comuna'] = df_cuar['Nombre'].apply(limpiar_nombre_comuna)

rm_comunas = [
    "Santiago", "Providencia", "Las Condes", "Vitacura", "Ñuñoa",
    "La Reina", "Macul", "Peñalolén", "La Florida", "Puente Alto",
    "Maipú", "Lo Barnechea", "Recoleta", "Cerrillos", "Lo Prado",
    "Pudahuel", "Quinta Normal", "San Miguel", "Estación Central",
    "La Cisterna", "San Joaquín", "El Bosque", "La Granja", "Pedro Aguirre Cerda",
    "San Ramón", "La Pintana", "San Bernardo", "Buin", "Calera de Tango", "Paine",
    "Pirque", "Padre Hurtado", "Renca", "Quilicura", "Independencia",
    "Cerro Navia", "Conchalí", "Lo Espejo", "Huechuraba", "Colina", "Lampa",
    "San José de Maipo", "Melipilla", "Curacaví", "Til Til", "Peñaflor", "Talagante",
    "Alhué", "María Pinto", "San Pedro", "Isla de Maipo", "El Monte"
]

df_cuar = df_cuar[df_cuar['Comuna'].isin(rm_comunas)]

df_cuar['Cuarentena_total_ManzanasInFull'] = df_cuar['ManzanasInFull'].fillna('').apply(lambda x: x.split())
df_cuar['Cuarentena_parcial_ManzanasInPartial'] = df_cuar['ManzanasInPartial'].fillna('').apply(lambda x: x.split())

df_agrupado = df_cuar.groupby('Comuna').agg({
    'Cuarentena_total_ManzanasInFull': lambda lists: list(set([m for sublist in lists for m in sublist])),
    'Cuarentena_parcial_ManzanasInPartial': lambda lists: list(set([m for sublist in lists for m in sublist]))
}).reset_index()

df_agrupado = df_agrupado.sort_values('Comuna').reset_index(drop=True)
df_agrupado.to_csv("cuarentenas_segun_codigo_manzanas_censo17.csv", index=False)
df_agrupado.head()


,Comuna,Cuarentena_total_ManzanasInFull,Cuarentena_parcial_ManzanasInPartial
0,Buin,"[13402011001067.0, 13402051002023.0, 134020110...","[13402011001067.0, 13402051002023.0, 134020110..."
1,Calera de Tango,"[13403051001031.0, 13403051001038.0, 134030510...","[13403051001031.0, 13403051001038.0, 134030510..."
2,Cerrillos,"[13102011003022.0, 13102031007002.0, 131020410...","[13102011003022.0, 13102031007002.0, 131020410..."
3,Cerro Navia,"[13103071005016.0, 13103021002017.0, 131030410...","[13103071005016.0, 13103021002017.0, 131030410..."
4,Colina,"[13301051001002.0, 13301011013004.0, 133010110...","[13301051001002.0, 13301011013004.0, 133010110..."


In [5]:
# Producto 38: Fallecidos por comuna

df = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto38/CasosFallecidosPorComuna.csv")

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_long = df_rm.melt(
    id_vars=["Region", "Codigo region", "Comuna", "Codigo comuna", "Poblacion"],
    var_name="Fecha",
    value_name="Casos fallecidos"
)

df_long["Fecha"] = pd.to_datetime(df_long["Fecha"], errors="coerce")

df_long["Año"] = df_long["Fecha"].dt.year

df_final = (df_long.groupby(["Comuna", "Año"], as_index=False)["Casos fallecidos"].sum().rename(columns={"Casos fallecidos": "Total fallecidos"}))

df_final = df_final[df_final["Comuna"].notna() & ~df_final["Comuna"].str.contains("Total|Desconocido", case=False)]

df_final.to_csv('muertes_por_anio_comunal.csv', index=False)
df_final.head()

# consuta a la IA: '¿Por qué me puede estar dando error el convertir unas fechas a pd.DateTime en csv?'
# Llevó a la implementación de melt


,Comuna,Año,Total fallecidos
0,Alhue,2020.0,83.0
1,Alhue,2021.0,541.0
2,Alhue,2022.0,1088.0
3,Alhue,2023.0,70.0
4,Buin,2020.0,1762.0


In [6]:
# Producto 41: transacciones BIP por comuna durante la pandemia

df = pd.read_csv('datos_covid_extraccion/datos-covid-19/producto41/BIPComuna_std.csv')

df['Fecha'] = pd.to_datetime(df['Fecha'], errors='coerce')

df['Año'] = df['Fecha'].dt.year

df_limpio = df.dropna(subset={'Comuna', 'Codigo comuna'})

df_final = (df_limpio.groupby(['Comuna', "Año"], as_index=False)['Transacciones'].sum())

df_final.to_csv('transacciones_bip_por_comuna.csv', index=False)
df_final.head(20)

,Comuna,Año,Transacciones
0,Cerrillos,2020,1616394.0
1,Cerro Navia,2020,1186714.0
2,Conchalí,2020,1641957.0
3,El Bosque,2020,926224.0
4,Estación Central,2020,5460067.0
5,Huechuraba,2020,776352.0
6,Independencia,2020,1503482.0
7,La Cisterna,2020,4298804.0
8,La Florida,2020,5380051.0
9,La Granja,2020,686554.0


In [7]:
# Producto 65: Positividas test covid por comuna

df = pd.read_csv('datos_covid_extraccion/datos-covid-19/producto65/PositividadPorComuna.csv')

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_long = df_rm.melt(
    id_vars=["Region", "Codigo region", "Comuna", "Codigo comuna", "Poblacion"],
    var_name="Fecha",
    value_name="Positividad"
)

df_long["Positividad"] = pd.to_numeric(df_long["Positividad"], errors="coerce")
df_long["Fecha"] = pd.to_datetime(df_long["Fecha"], errors="coerce")
df_long["Año"] = df_long["Fecha"].dt.year

df_long = df_long.drop(columns={"Codigo region", "Codigo comuna", "Fecha"})

df_final = (df_long.groupby(['Comuna', "Año"], as_index=False)['Positividad'].sum())

df_final.to_csv('positividad_test_covid19.csv', index=False)
df_final.head()

,Comuna,Año,Positividad
0,Alhue,2020,99.0
1,Alhue,2021,271.4
2,Alhue,2022,610.0
3,Buin,2020,90.1
4,Buin,2021,385.7


In [8]:
# Producto 66: cobertura de casos sospechosos con test

df = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto66/CoberturaPorComuna_std.csv")

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_rm["Cobertura testeo"] = pd.to_numeric(df_rm["Cobertura testeo"], errors="coerce")
df_rm["Fecha"] = pd.to_datetime(df_rm["Fecha"], errors="coerce")
df_rm["Año"] = df_rm["Fecha"].dt.year

df_promedio = (
    df_rm.groupby(["Comuna", "Año"], as_index=False)["Cobertura testeo"]
    .mean()
    .rename(columns={"Cobertura testeo": "Cobertura promedio"})
)

df_promedio.to_csv('cobertura_casos_sospechosos_con_test_prom.csv', index=False)
df_promedio.head()


,Comuna,Año,Cobertura promedio
0,Alhue,2020,89.584211
1,Alhue,2021,67.358824
2,Alhue,2022,88.466667
3,Buin,2020,90.894737
4,Buin,2021,88.646154


In [9]:
# Archivo 80: Avance primera dosis vacuna covid-19

df = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto80/vacunacion_comuna_1eraDosis_std.csv")

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_rm["Primera Dosis"] = pd.to_numeric(df_rm["Primera Dosis"], errors="coerce")
df_rm["Fecha"] = pd.to_datetime(df_rm["Fecha"], errors="coerce")
df_rm["Año"] = df_rm["Fecha"].dt.year

df_primera_dosis = (df_rm.groupby(["Comuna", "Año"], as_index=False)["Primera Dosis"].sum())

df_primera_dosis.rename(columns={"Primera Dosis": "Vacunados 1ra dosis"}, inplace=True)

df_primera_dosis

,Comuna,Año,Vacunados 1ra dosis
0,Alhue,2020,0.0
1,Alhue,2021,7894.0
2,Alhue,2022,551.0
3,Alhue,2023,5.0
4,Buin,2020,0.0
...,...,...,...
207,Tiltil,2023,11.0
208,Vitacura,2020,337.0
209,Vitacura,2021,122074.0
210,Vitacura,2022,3332.0


In [10]:
# Producto 80: vacunados segunda dosis

df = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto80/vacunacion_comuna_2daDosis_std.csv")

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_rm["Segunda Dosis"] = pd.to_numeric(df_rm["Segunda Dosis"], errors="coerce")
df_rm["Fecha"] = pd.to_datetime(df_rm["Fecha"], errors="coerce")
df_rm["Año"] = df_rm["Fecha"].dt.year

df_segunda_dosis = (df_rm.groupby(["Comuna", "Año"], as_index=False)["Segunda Dosis"].sum())

df_segunda_dosis.rename(columns={"Segunda Dosis": "Vacunados 2da dosis"}, inplace=True)

df_segunda_dosis

,Comuna,Año,Vacunados 2da dosis
0,Alhue,2020,0.0
1,Alhue,2021,7538.0
2,Alhue,2022,666.0
3,Alhue,2023,1.0
4,Buin,2020,0.0
...,...,...,...
207,Tiltil,2023,2.0
208,Vitacura,2020,0.0
209,Vitacura,2021,118621.0
210,Vitacura,2022,5873.0


In [11]:
# Producto 80: cuarta dosis cant vacunados

df = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto80/vacunacion_comuna_4taDosis_std.csv")

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_rm["Cuarta Dosis"] = pd.to_numeric(df_rm["Cuarta Dosis"], errors="coerce")
df_rm["Fecha"] = pd.to_datetime(df_rm["Fecha"], errors="coerce")
df_rm["Año"] = df_rm["Fecha"].dt.year

df_cuarta_dosis = (df_rm.groupby(["Comuna", "Año"], as_index=False)["Cuarta Dosis"].sum())

df_cuarta_dosis.rename(columns={"Cuarta Dosis": "Vacunados 4ta dosis"}, inplace=True)

df_cuarta_dosis

,Comuna,Año,Vacunados 4ta dosis
0,Alhue,2020,0.0
1,Alhue,2021,0.0
2,Alhue,2022,6052.0
3,Alhue,2023,30.0
4,Buin,2020,0.0
...,...,...,...
207,Tiltil,2023,61.0
208,Vitacura,2020,0.0
209,Vitacura,2021,0.0
210,Vitacura,2022,98643.0


In [12]:
# Producto 80: dosis refuerzo cant vacunados

df = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto80/vacunacion_comuna_Refuerzo_std.csv")

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_rm["Dosis Refuerzo"] = pd.to_numeric(df_rm["Dosis Refuerzo"], errors="coerce")
df_rm["Fecha"] = pd.to_datetime(df_rm["Fecha"], errors="coerce")
df_rm["Año"] = df_rm["Fecha"].dt.year

df_dosis_refuerzo = (df_rm.groupby(["Comuna", "Año"], as_index=False)["Dosis Refuerzo"].sum())

df_dosis_refuerzo.rename(columns={"Dosis Refuerzo": "Vacunados dosis de refuerzo"}, inplace=True)

df_dosis_refuerzo

,Comuna,Año,Vacunados dosis de refuerzo
0,Alhue,2020,0.0
1,Alhue,2021,5430.0
2,Alhue,2022,2516.0
3,Alhue,2023,7.0
4,Buin,2020,0.0
...,...,...,...
207,Tiltil,2023,16.0
208,Vitacura,2020,0.0
209,Vitacura,2021,96883.0
210,Vitacura,2022,19698.0


In [13]:
# Producto 80: unica dosis cant vacunados

df = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto80/vacunacion_comuna_UnicaDosis_std.csv")

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_rm["Unica Dosis"] = pd.to_numeric(df_rm["Unica Dosis"], errors="coerce")
df_rm["Fecha"] = pd.to_datetime(df_rm["Fecha"], errors="coerce")
df_rm["Año"] = df_rm["Fecha"].dt.year

df_unica_dosis = (df_rm.groupby(["Comuna", "Año"], as_index=False)["Unica Dosis"].sum())

df_unica_dosis.rename(columns={"Unica Dosis": "Vacunados unica dosis"}, inplace=True)

df_unica_dosis

,Comuna,Año,Vacunados unica dosis
0,Alhue,2020,0.0
1,Alhue,2021,602.0
2,Alhue,2022,0.0
3,Alhue,2023,0.0
4,Buin,2020,0.0
...,...,...,...
207,Tiltil,2023,0.0
208,Vitacura,2020,0.0
209,Vitacura,2021,2179.0
210,Vitacura,2022,0.0


In [14]:
# Producto 80: vacunación total por comuna

df_vacunas = (
    df_primera_dosis
    .merge(df_segunda_dosis, on=["Comuna", "Año"], how="outer")
    .merge(df_dosis_refuerzo, on=["Comuna", "Año"], how="outer")
    .merge(df_cuarta_dosis, on=["Comuna", "Año"], how="outer")
    .merge(df_unica_dosis, on=["Comuna", "Año"], how="outer")
)

df_vacunas = df_vacunas.fillna(0)

df_vacunas.to_csv('cant_vacunados_distintas_dosis.csv', index=False)
df_vacunas.head(20)


,Comuna,Año,Vacunados 1ra dosis,Vacunados 2da dosis,Vacunados dosis de refuerzo,Vacunados 4ta dosis,Vacunados unica dosis
0,Alhue,2020,0.0,0.0,0.0,0.0,0.0
1,Alhue,2021,7894.0,7538.0,5430.0,0.0,602.0
2,Alhue,2022,551.0,666.0,2516.0,6052.0,0.0
3,Alhue,2023,5.0,1.0,7.0,30.0,0.0
4,Buin,2020,0.0,0.0,0.0,0.0,0.0
5,Buin,2021,98241.0,93820.0,63672.0,0.0,2613.0
6,Buin,2022,3959.0,6178.0,27184.0,67680.0,0.0
7,Buin,2023,37.0,35.0,88.0,272.0,0.0
8,Calera de Tango,2020,0.0,0.0,0.0,0.0,0.0
9,Calera de Tango,2021,28409.0,27165.0,20076.0,0.0,1746.0


In [15]:
# Producto 84 parte 1: fallecidos 2020-2021

df = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto84/fallecidos_comuna_edad_totales_std_A.csv")

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_rm["Total"] = pd.to_numeric(df_rm["Total"], errors="coerce")
df_rm["Fecha"] = pd.to_datetime(df_rm["Fecha"], errors="coerce")
df_rm["Año"] = df_rm["Fecha"].dt.year

df_muertes_a = (df_rm.groupby(["Comuna", "Año"], as_index=False)["Total"].sum().rename(columns={"Total": "Fallecidos"}))

df_muertes_a.head()


,Comuna,Año,Fallecidos
0,Alhue,2020,6.0
1,Alhue,2021,8.0
2,Alhue,2022,4.0
3,Buin,2020,158.0
4,Buin,2021,132.0


In [16]:
# Producto 84 parte 2: total fallecidos 2022

df = pd.read_csv("datos_covid_extraccion/datos-covid-19/producto84/fallecidos_comuna_edad_totales_std_B.csv")

df_rm = df[df["Region"].str.contains("Metropolitana", case=False, na=False)].copy()

df_rm["Total"] = pd.to_numeric(df_rm["Total"], errors="coerce")
df_rm["Fecha"] = pd.to_datetime(df_rm["Fecha"], errors="coerce")
df_rm["Año"] = df_rm["Fecha"].dt.year

df_muertes_b = (df_rm.groupby(["Comuna", "Año"], as_index=False)["Total"].sum().rename(columns={"Total": "Fallecidos"}))

df_muertes_b.head()


,Comuna,Año,Fallecidos
0,Alhue,2022,4.0
1,Buin,2022,19.0
2,Calera de Tango,2022,6.0
3,Cerrillos,2022,26.0
4,Cerro Navia,2022,41.0


In [17]:
# Producto 84 final: Muertes 2020-2022

df_muertes = pd.concat([df_muertes_a, df_muertes_b], ignore_index=True)

df_muertes = df_muertes.groupby(["Comuna", "Año"], as_index=False)["Fallecidos"].sum()

df_muertes.to_csv('cant_muertes_20-21-22.csv', index=False)
df_muertes.head()


,Comuna,Año,Fallecidos
0,Alhue,2020,6.0
1,Alhue,2021,8.0
2,Alhue,2022,8.0
3,Buin,2020,158.0
4,Buin,2021,132.0
